In [131]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
from collections import defaultdict

from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font


def check_master(song_title, index_check=False):
    substrings = ['Master','Mast']
    if not index_check:
        return any([substring.lower() in song_title.lower() for substring in substrings])
    else:
        return max([song_title.lower().find(substring.lower()) for substring in substrings])

def create_output(copyright_dict, name):
    wb =  Workbook()
    ws =  wb.active
    headings = ["#","Song","Registration #"]
    ws.append(headings)
    
    for key in copyright_dict.keys():
        #char = get_column_letter(col)
        # easy mvp but need to store original row number and do something like ws[char + row_iter]
        row_num = key[0]
        song_title = key[1]
        copy_num = copyright_dict[key]
        ws.append([row_num, song_title, copy_num])
    
    wb.save(name + '.xlsx')
    print("Check out the output file!")
    
if __name__ == "__main__":
    
    require_cols = [2, 2]
    dataframe1 = pd.read_excel('/Users/xiaoyanyang/Desktop/MusicCatalog_WebCrawler/input_Song_Titles.xlsx', engine = 'openpyxl', usecols = require_cols, skiprows = 2, header = None).head(-142)
    copyright_dict = defaultdict(str)
    
    song_titles = []
    #breaks_code = ["I'M TOO SEXY", 'HUMBLE AND KIND (TIM MCGRAW MA', 'HOLIDAY (SSH)', "CAN'T YOU SEE (U.S. ONLY AS OF)", "HIGHWAY DON'T CARE (TIM MCGRAW", '25 OR 6 TO 4 (GOARMY REMIX)', 'DANCE HALL DAYS (WANG CHUNG RE', 'SHOTGUN RIDER (TIM MCGRAW MAST', 'SHOTGUN RIDER (TIM MCGRAW MAST', 'BAREFOOT BLUE JEAN NIGHT (BMI']
    breaks_code = ["I'M TOO SEXY", 'LET MY LOVE OPEN THE DOOR', "CAN'T YOU SEE (U.S. ONLY AS OF)", "WON'T GET FOOLED AGAIN", "WON'T GET FOOLED AGAIN", 'LOOK WHAT YOU MADE ME DO']
    
    for index, row in dataframe1.iterrows():
        if row[2].split("-")[1] not in breaks_code:
            if "(" in row[2]:
                first_half = row[2].split("(")[0]
                song_titles.append((first_half.split("-"))[1])
            else:
                song_titles.append((row[2]).split("-")[1])
    
    PATH = "/Users/xiaoyanyang/Desktop/chromedriver"
    driver = webdriver.Chrome(PATH)
    driver.get("https://cocatalog.loc.gov/cgi-bin/Pwebrecon.cgi?DB=local&PAGE=first")
    #song_titles = ["20th Century Boy", "Burn Out", "Glorious Domination (Master)", "More Girls Like You", "Stars in the night"]
    #song_titles = ["Glorious Domination (Master)"]
    #song_titles = ["CAN'T YOU SEE (U.S. ONLY AS OF)"]
    #song_title = ["HUMBLE AND KIND (TIM MCGRAW MA"]

    #if "(" in song_title[0]:
    #    first_half = row[2].split("(")[0]
    #    song_titles.append((first_half.split("-"))[1])
    
    count = 0
    for i in range(len(song_titles)):
        count += 1
        song_title = song_titles[i]
        if check_master(song_title):
            end_index = check_master(song_title, True)
            song_title = song_title[:end_index-1]
        
        # search_term can be song title + catalog (try different search terms)
        
        search = driver.find_element_by_name("Search_Arg")
        search.clear()
        search.send_keys(song_title + "\n")

        driver.implicitly_wait(0.5)
        
        possibilities_list = []
        
        try:
            table = driver.find_elements_by_xpath("//form/table[2]/tbody/tr")
        except:
            table = driver.find_elements_by_xpath("//form/table[2]")
            
        num_cols = len(table[1].text.split(":"))
        #print("number of cols is", num_cols)
        if num_cols == 2: # details page of single search result case
            row_item = table[2].text
            #print("row item is", row_item)
            try: 
                copy_num = row_item.split(":")[-1].split("/")[-2]
            except:
                # /html/body/form[1]/table[2]/tbody/tr[3]/th
                # need to figure out how to access the column over (while keeping in mind that registration number can show up in different rows)
                row_index = driver.find_elements_by_xpath("//*[contains(text(), 'Registration Number')]").index
                #print(row_index)
            possibilities_list.append(copy_num) 
            
        else: # more than one search result case
            if not i:
                sel = Select(driver.find_element_by_xpath("//select[@name='CNT']"))
                sel.select_by_value("100")
                driver.implicitly_wait(0.5)
                submit_button = driver.find_element_by_xpath("//form/center[2]/form/table/tbody/tr[2]/td/div/div/table/tbody/tr/td[2]/div/input[2]")
                submit = ActionChains(driver)
                submit.click(submit_button)
                submit.perform()
            
            try:
                table = driver.find_elements_by_xpath("//form/table[2]/tbody/tr")
            except:
                table = driver.find_elements_by_xpath("//form/table[2]")
            num_rows = len(table)
            for i in range(1,num_rows):
                row_item = table[i].text #
                copy_num = row_item.split(".")[-1].split(" ")[-2]
                song_info = row_item
                if (song_title.lower() in song_info.lower()):
                    date = row_item.split(".")[-1].split(" ")[-1]
                    if check_master(song_title) and copy_num[:2] == "SR":
                        # possibilities_list.append((copy_num, date)) 
                        # instead of appending date, in better MVP would click into details page 
                        #     and see whether catalog name is listed in authorship on application and append type of work 
                        possibilities_list.append(copy_num)

                    elif copy_num[:2] == "PA" and copy_num[:3] != "PAu": 
                        possibilities_list.append(copy_num) 
        
        print(count)
        print(song_title)
        print(possibilities_list, "\n")
        
        #{(song name, row num):copyright #} dictionary
        # in the excel file row num offset by +2
        if not possibilities_list:
            copyright_dict[(count, song_title)] = " "
        else:
            copyright_dict[(count, song_title)] = possibilities_list[0]
        
        try:
            search_toggle = driver.find_element_by_xpath("//center[2]/font/a[2]/img")
        except:
            search_toggle = driver.find_element_by_xpath("//center[2]/font/img")
        finally:
            back_tosearch = ActionChains(driver)
            back_tosearch.click(search_toggle)
            back_tosearch.perform()
            
    driver.close()
    print(copyright_dict) 
    
    create_output(copyright_dict, "Registration Numbers")

# next steps: 
# - paste to excel file (just take first element of list)
# - use more complete information of excel sheet (catalog name, artist name, date)
# - figure out how to seek registration number dynamically within details page of song
# turn into object-oriented so that things like copyright_dict are global
# build a web interface, start the process after uploading input file and pressing button 
# fix 20th century boy Master version (don't split at the beginning right away on "(" if detect Master)

1
20th Century Boy
['PA0000976753'] 

2
WHO ARE YOU
['PA0001864711', 'PA0002172409', 'PA0002024029', 'PA0002214781', 'PA0001923507', 'PA0000971407', 'PA0001007374', 'PA0001188039', 'PA0000978896'] 

3
BABA O'RILEY
['PA0000720226'] 

4
MEANT TO BE
['PA0001751784', 'PA0001753919', 'PA0000368912', 'PA0001848514', 'PA0001754717', 'PA0001891926', 'PA0001921758', 'PA0000527016', 'PA0001007504'] 

5
HUMBLE AND KIND 
['PA0002127076'] 

6
SOUTHBOUND
['PA0002267459', 'PA0000008479', 'PA0002152846', 'PA0002029830', 'PA0001715837', 'PA0001149892'] 

7
BEER NEVER BROKE MY HEART
['PA0002192567', 'PA0002233234'] 

8
HOLIDAY 
['PA0001041153', 'PA0000846204'] 

9
20th Century Boy 
[] 

10
Heaven Is A Place On Earth
['PA0000877899', 'PA0000488505'] 

11
BEHIND BLUE EYES
['PA0000347339', 'PA0001226899', 'PA0000729885', 'PA0000412014'] 

12
PINBALL WIZARD
['PA0000750495', 'PA0002206487', 'PA0002103556', 'PA0001007246', 'PA0001828904'] 

13
Children Of The Revolution
['PA0000536588', 'PA0001015278', 'PA000

In [126]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

def create_output():
    copyright_dict = {(1, '20th Century Boy'): 'V3613D402', (2, 'WHO ARE YOU'): 'SRu000116460', (3, "BABA O'RILEY"): 'SR0000776140', (4, 'MEANT TO BE'): 'SRu000868462', (5, 'HUMBLE AND KIND '): 'SR0000814718', (6, 'LET MY LOVE OPEN THE DOOR'): '[ 2 ] Let my love open the door -- Empty glass ', (7, 'SOUTHBOUND'): 'PA0001149892', (8, 'BEER NEVER BROKE MY HEART'): 'SR0000863063', (9, 'HOLIDAY '): 'RE0000269852', (10, '20th Century Boy '): 'V3613D402', (11, "WON'T GET FOOLED AGAIN"): "[ 2 ] Won't get fooled again. Beautiful loser, dirty white boy ", (12, 'Heaven Is A Place On Earth'): 'TX0005948879', (13, 'BEHIND BLUE EYES'): 'SR0000802916', (14, 'PINBALL WIZARD'): 'PAu003773594', (15, 'Children Of The Revolution'): 'PA0001059197', (16, "HIGHWAY DON'T CARE "): 'V9942D059', (17, 'LOVE WINS'): 'V3603D994', (18, 'GONNA MAKE YOU SWEAT'): 'V3615D973', (19, 'TEACH YOUR CHILDREN'): 'V3615D973', (20, 'You Get What You Give'): 'V3615D973', (21, 'DRINKING ALONE'): 'VA0002145857', (22, 'BODY LIKE A BACK ROAD'): 'SR0000806425', (23, '25 OR 6 TO 4 '): 'V3510D162', (24, 'MY GENERATION'): 'PAu003672704', (25, 'THIS IS IT'): 'SR0000013514', (26, 'DANCE HALL DAYS '): 'PA0000259814', (27, 'THE SEEKER'): 'TX0004936830', (28, 'THE WAY I AM'): 'SR0000639610', (29, 'JOIN TOGETHER'): 'PAu002161082', (30, 'CHA CHA SLIDE'): 'PA0001066911', (31, 'SHOTGUN RIDER '): 'TXu001871010', (32, 'DANCE HALL DAYS'): 'PA0000259814', (33, 'BODY'): 'SRu001075749', (34, 'TOMMY '): 'VAu000525983', (35, "I'M A BOY"): 'SR0000830141', (36, 'EVERYBODY HAVE FUN TONIGHT'): 'PA0000316303', (37, 'CHRISTMAS WRAPPING'): 'VA0000884159', (38, 'HERE AND NOW'): 'SRu001415475', (39, 'FAMILY AFFAIR '): 'SRu001415475', (40, 'BAREFOOT BLUE JEAN NIGHT '): 'V3618D443', (41, 'SUNRISE SUNBURN SUNSET'): 'VA0002130667', (42, 'GOOD VIBES'): 'V3441D410', (43, "RIDIN' ROADS"): 'SR0000845217', (44, "I HOPE YOU'RE HAPPY NOW"): 'V3499D906', (45, "LET'S TWIST AGAIN"): 'TX0002685123', (46, 'I Know You Want Me'): 'TX0002685123', (47, 'EMINENCE FRONT'): 'PAu002848829'}
    wb =  Workbook()
    ws =  wb.active
    headings = ["#","Song","Registration #"]
    ws.append(headings)
    
    for ((row_num, song_title), copy_num) in copyright_dict.items():
        #char = get_column_letter(col)
        # easy mvp but need to store original row number and do something like ws[char + row_iter]
        ws.append([row_num, song_title, copy_num])
    
    wb.save('NewGrades.xlsx')

if __name__ == "__main__":
    create_output()

In [130]:
lst = []
print(not lst)

lst1 = [2]
print(not lst1)

True
False


In [51]:
!pip3 install openpyxl

In [95]:
import pandas as pd
  
require_cols = [2, 2]
#new_columns = ["Row Index", "Song Title"]
  
dataframe1 = pd.read_excel('/Users/xiaoyanyang/Desktop/MusicCatalog_WebCrawler/input_Song_Titles.xlsx', engine = 'openpyxl', usecols = require_cols, skiprows = 2, header = None).head(-142)

print(dataframe1)
#print(list(dataframe1.columns))

                                        2
0                  13463-20th Century Boy
1                       54307-WHO ARE YOU
2                      56418-BABA O'RILEY
3                      191848-MEANT TO BE
4                      12279-I'M TOO SEXY
5   202775-HUMBLE AND KIND (TIM MCGRAW MA
6         54222-LET MY LOVE OPEN THE DOOR
7                       191897-SOUTHBOUND
8        202993-BEER NEVER BROKE MY HEART
9                       987-HOLIDAY (SSH)
10  24776-CAN'T YOU SEE (U.S. ONLY AS OF)
11        19819-20th Century Boy - Master
12           56402-WON'T GET FOOLED AGAIN
13       11872-Heaven Is A Place On Earth
14                 56421-BEHIND BLUE EYES
15                   56531-PINBALL WIZARD
16       13262-Children Of The Revolution
17  202740-HIGHWAY DON'T CARE (TIM MCGRAW
18                       191840-LOVE WINS
19        159744-LOOK WHAT YOU MADE ME DO
20              1943-GONNA MAKE YOU SWEAT
21             100056-TEACH YOUR CHILDREN
22            12008-You Get What Y

In [100]:
song_titles = []
for index, row in dataframe1.iterrows():
    song_titles.append((row[2]).split("-")[1])
print(song_titles)

['20th Century Boy', 'WHO ARE YOU', "BABA O'RILEY", 'MEANT TO BE', "I'M TOO SEXY", 'HUMBLE AND KIND (TIM MCGRAW MA', 'LET MY LOVE OPEN THE DOOR', 'SOUTHBOUND', 'BEER NEVER BROKE MY HEART', 'HOLIDAY (SSH)', "CAN'T YOU SEE (U.S. ONLY AS OF)", '20th Century Boy ', "WON'T GET FOOLED AGAIN", 'Heaven Is A Place On Earth', 'BEHIND BLUE EYES', 'PINBALL WIZARD', 'Children Of The Revolution', "HIGHWAY DON'T CARE (TIM MCGRAW", 'LOVE WINS', 'LOOK WHAT YOU MADE ME DO', 'GONNA MAKE YOU SWEAT', 'TEACH YOUR CHILDREN', 'You Get What You Give', 'DRINKING ALONE', 'BODY LIKE A BACK ROAD', '25 OR 6 TO 4 (GOARMY REMIX)', 'MY GENERATION', 'THIS IS IT', 'DANCE HALL DAYS (WANG CHUNG RE', 'THE SEEKER', 'THE WAY I AM', 'JOIN TOGETHER', 'CHA CHA SLIDE', 'SHOTGUN RIDER (TIM MCGRAW MAST', 'DANCE HALL DAYS', 'BODY', 'TOMMY ', "I'M A BOY", 'EVERYBODY HAVE FUN TONIGHT', 'CHRISTMAS WRAPPING', 'HERE AND NOW', 'FAMILY AFFAIR (BOTH SHARES)', 'BAREFOOT BLUE JEAN NIGHT (BMI', 'SUNRISE SUNBURN SUNSET', 'GOOD VIBES', "RIDIN' 

In [96]:
import openpyxl.utils.dataframe 
from openpyxl import Workbook, load_workbook
from itertools import islice
import pandas as pd

wb = Workbook()
wb =  load_workbook('/Users/xiaoyanyang/Desktop/MusicCatalog_WebCrawler/input_Song_Titles.xlsx')
ws = wb.active
data = ws.values

In [76]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time

In [95]:
PATH = "/Users/xiaoyanyang/Desktop/chromedriver"
driver = webdriver.Chrome(PATH)
driver.get("https://cocatalog.loc.gov/cgi-bin/Pwebrecon.cgi?DB=local&PAGE=first")

In [96]:
search = driver.find_element_by_name("Search_Arg")
search.clear()
search.send_keys("20th Century Boy\n")

In [97]:
driver.implicitly_wait(0.5)
time.sleep(0.5)
#time.sleep(5) 
#try:
    #table = WebDriverWait(driver, 10).until(
        #EC.presence_of_element_located((By.LINK_TEXT, "Beginner Python Tutorials"))
#    )
sel = Select(driver.find_element_by_xpath("//select[@name='CNT']"))
sel.select_by_value("100")
time.sleep(0.8)
submit_button = driver.find_element_by_xpath("//form/center[2]/form/table/tbody/tr[2]/td/div/div/table/tbody/tr/td[2]/div/input[2]").click() 
submit = ActionChains(driver)
submit.click(submit_button)
submit.perform()

In [98]:
table = driver.find_elements_by_xpath("//form/table[2]/tbody/tr")
num_rows = len(table)
print("num of rows is: ", num_rows)
for i in range(num_rows):
    row_item = table[i].text
    print(row_item)

num of rows is:  32
# Title < Full Title Copyright Number Date
[ 1 ] 20th Century Boy. 20th Century Boy. V3615D975 2012
[ 2 ] 20th Century Boy. 20th Century Boy. V3615D975 2012
[ 3 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3598D036 2011
[ 4 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3598D037 2011
[ 5 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3598D044 2011
[ 6 ] 20th Century Boy. 20th Century Boy. PAu003381057 2009
[ 7 ] 20th century boy. 20th century boy. V3582D425 2009
[ 8 ] 20th century boy. 20th century boy. V3576D005 2009
[ 9 ] 20th century boy. 20th century boy. V3576D006 2009
[ 10 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3602D215 2009
[ 11 ] 20th century boy. 20th century boy. V3562D695 2008
[ 12 ] 20th century boy. 20th century boy. V3562D696 2008
[ 13 ] 20th century boy. 20th century boy. PA0000976753 1999
[ 14 ] 20th century boy 20th century boy ; Dirt / Lorette Velvette. SR0000233398 1995
[ 15 ] 20th century boy -- Ain't no jive, live! / Bang Tango. SR0000143711 1992
[ 16 ] 20th century 

In [128]:
def check_master(song_title):
    substrings = ['Master','Mast']
    #song_title = '20th Century Boy - MASTER'

    output = any([substring.lower() in song_title.lower() for substring in substrings])
    #print(output)
    return output

In [130]:
num_rows = len(table)
song_title = "20th Century Boy"
print("num of rows is: ", num_rows)
possibilities_list = []
for i in range(1,num_rows):
    row_item = table[i].text
    copy_num = row_item.split(".")[-1].split(" ")[-2]
    song_info = row_item
    #print(song_info)
    if (song_title.lower() in song_info.lower()):
        date = row_item.split(".")[-1].split(" ")[-1]
        if check_master(song_title) and copy_num[:2] == "SR":
            possibilities_list.append((copy_num, date)) 
            
        elif copy_num[:2] == "PA": 
            possibilities_list.append((copy_num, date)) 
            #print("True")
            #print(copy_num)
            #print(row_item.split("."))
    #print(row_item)
print(possibilities_list)
    

num of rows is:  32
[('PAu003381057', '2009'), ('PA0000976753', '1999')]


In [13]:
substrings = ['a', 'b', 'c']
a_string = 'cde'

print(substrings)
print(a_string)

output = any([substring in a_string for substring in substrings])
print(output)

['a', 'b', 'c']
cde
True


In [24]:
substrings = ["Master", "Mast"]
a_string = "Glorious Domination (Master)"

output = [a_string.lower().find(substring.lower()) for substring in substrings]
print(output)
print(a_string[:max(output)-1])

[21, 21]
Glorious Domination 


In [92]:
search_toggle = driver.find_element_by_xpath("//center[2]/font/a[2]/img")
back_tosearch = ActionChains(driver)
back_tosearch.click(search_toggle)
back_tosearch.perform()

In [50]:
xpath1 = "//form/table[2]/tbody/tr["+str(2)+"]/td[3]"
full_title = driver.find_element_by_xpath(xpath1).text
print(row_item)

[ 25 ] 20th Century boy. PA 976-753. 20th Century boy. PA 976-753. V3570D349 2008


In [43]:
row_item = driver.find_element_by_xpath("//form/table[2]/tbody/tr[11]/td[4]").text
print(row_item)

V3602D215


In [ ]:
num_rows = len(table)
print("num of rows is: ", num_rows)
for i in range(num_rows):
    xpath1 = "//form/table[2]/tbody/tr["+str(i)+"]/td[3]"
    full_title = driver.find_element_by_xpath(xpath1).text
    xpath2 = "//form/table[2]/tbody/tr["+str(i)+"]/td[4]"
    copy_num = driver.find_element_by_xpath(xpath2).text
    print(full_title, copy_num)
#print("num of columns is: ", len(table[0].split(".")))

### TODO's

MVP: Where only the song name & catalog name is provided, not the date & where we only look at the first 100 search results & we take only the first PA or SR copy # that shows up (no other types of copyright numbers and no considerations for the details page to determine which copyright num is best if multiple possibilities)
---> test against manually inputted data
MVP child: multiple possibilities user prompt
MVP grand-child: google search date of the song & cross-compare for higher accuracy
MVP grand-grand-child: Streamlit interface for webpage: upload file button for excel file and download new Excel file  

1. make ActionChain of search action
2. create dummy CSV of song names & ids
3. iterate through song names & ids and mark down the row number of where we wanna paste the copyright # to
- driver to click on the search button (back to search screen) in between searches 
4. for each song name, do the whole ActionChain (wrap search into ActionChain object), jot down the copyright # in the {(song name, row num):copyright #} dictionary
    - count = 0
    - while count < 3 and !len(possibilities)
        - if condition: if the text saying "displaying # through # of # entries) has the middle # less than the last #, click on the NEXT arrow button 
        - count += 1
    - for cases where there is MASTER or MAST in the title, check for SR only
        - case MULTIPLE
    - else, check for PA only
        - case MULTIPLE
    - for cases where there are no PA or SR, look <-- eh, 
    - case MULTIPLE: for cases where there are multiple PA's or SR's, don't record in dictionary right away
        - come up with list of possibilities by clicking into the page details of a particular song entry if the copyright num is PA for non-MASTER case or SR if MASTER case
            - save down the Type of Work info
        - while user_input != "Y" and count < len(possibilities):
            - Info for this song:
            - Song Name / Catalog / Type of Work:
            - prompt user: "accept " + str(copy_num) + "copyright number, Y/N: "            
        

5. Output: 
    - in python: print out how many songs out of the song titles passed in were registered with PA or SR copyright num (# out of # song titles registered)
    - MVP: 
        - input: just a single column with the id - song title entries
        - output: fill in the adjacent column to the song title column 
    - MVP-child: in excel: fill in the appropriate cells based on the copyright nums recorded

misc. 
- may take a while so do multiple passes
    - 1st pass: only look at first page of search results
- create a summary table of the song titles with master in the name and the songs without